In [ ]:
import json
with open('path/to/scienceqa/train.json','r') as f:
    data = json.load(f)

with open('path/to/scienceqa/test.json','r') as f:
    data_test = json.load(f)

In [ ]:
from PIL import Image as PILImage

vqa = []

for item in data:
    if 'image' in item.keys():
        problem = item['conversations'][0]['value']
        images = [PILImage.open(item['image'].replace('ScienceQA/images/train','path/to/your/images/train'))]
        answer = item['conversations'][-1]['value']
        vqa.append({
            'images': images,
            'problem': problem,
            'answer': answer
        })

vqa_test = []

for item in data_test:
    if 'image' in item.keys():
        problem = '<image>\n'+item['text'].replace('Context: ','')
        images =[PILImage.open(item['image'].replace('ScienceQA/images/test','/home/esg8sdce/esg8sdceuser01/project/mllm_cl/tasks/ScienceQA/images/test'))]
        answer = item['answer']
        vqa_test.append({
            'images': images,
            'problem': problem,
            'answer': answer
        })


In [3]:
vqa[:5]

[{'images': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429>],
  'problem': "<image>\nWhich of these states is farthest north?\nA. West Virginia\nB. Louisiana\nC. Arizona\nD. Oklahoma\nAnswer with the option's letter from the given choices directly.",
  'answer': 'A'},
 {'images': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=302x232>],
  'problem': "<image>\nContext: The passage below describes an experiment. Read the passage and then follow the instructions below.\n\nTom placed a ping pong ball in a catapult, pulled the catapult's arm back to a 45° angle, and launched the ball. Then, Tom launched another ping pong ball, this time pulling the catapult's arm back to a 30° angle. With each launch, his friend Justin measured the distance between the catapult and the place where the ball hit the ground. Tom and Justin repeated the launches with ping pong balls in four more identical catapults. They compared the distances the balls traveled when launched from a 45° an

In [ ]:
from datasets import Dataset, Features, Image, Value
from PIL import Image as PILImage
import os


def process_data(vqa_data):
    images = []
    problems = []
    answers = []

    for example in vqa_data:
        problems.append(example['problem'])
        answers.append(example['answer'])
        
        if 'images' in example:
            images.append(example['images'])
        else:
            images.append(None)  # 没有图像时，存储 None 或跳过

    return {
        'images': images,
        'problem': problems,
        'answer': answers
    }

# 处理数据
processed_data = process_data(vqa)
processed_data_test = process_data(vqa_test)
# 定义数据集的 features
features = Features({
    "images": [Image(mode=None, decode=True)],  # 图像字段是一个列表，每个元素是一个 PIL 图像对象
    "problem": Value(dtype="string"),          # 问题字段，字符串类型
    "answer": Value(dtype="string")            # 答案字段，字符串类型
})

# 创建数据集
train_set = Dataset.from_dict(processed_data, features=features)
test_set = Dataset.from_dict(processed_data_test, features=features)


/home/esg8sdce/esg8sdceuser01/miniconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
train_set[3]

{'images': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=251x239>],
 'problem': "<image>\nContext: This passage describes the myotonia congenita trait in goats:\nMyotonia congenita is a condition that causes temporary muscle stiffness. When goats with myotonia congenita attempt to run from a resting position, their leg muscles often stiffen, causing them to fall over. Because of this behavior, these goats are referred to as fainting goats. Myotonia congenita is also found in other mammals, including horses, cats, and humans.\nIn a group of goats, some individuals have myotonia congenita and others do not. In this group, the gene for the myotonia congenita trait has two alleles. The allele for having myotonia congenita (M) is dominant over the allele for not having myotonia congenita (m).\nThis Punnett square shows a cross between two goats.\nWhat is the probability that a goat produced by this cross will be homozygous dominant for the myotonia congenita gene?\nA. 1/4\nB. 0/4\nC

In [ ]:
from datasets import DatasetDict
dataset1 = DatasetDict({
    "train": train_set,
    "test": test_set
})

In [5]:
from datasets import DatasetDict
dataset1 = DatasetDict({
    "train": train_set,
    "test": test_set
})


In [5]:
dataset1['train'][0]

{'images': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429>],
 'problem': "<image>\nWhich of these states is farthest north?\nA. West Virginia\nB. Louisiana\nC. Arizona\nD. Oklahoma\nAnswer with the option's letter from the given choices directly.",
 'answer': 'A'}

In [6]:
import os
output_dir = "/home/esg8sdce/esg8sdceuser01/project/mllm_cl/cl_datasets/ScienceQA"
os.makedirs(output_dir, exist_ok=True)

# 保存每个split为parquet格式
for split, dataset in dataset1.items():
    file_name = f"{split}-00000-of-00001.parquet"
    output_path = os.path.join(output_dir, file_name)
    
    # 将数据集保存为parquet格式
    dataset.to_parquet(output_path)

print("数据集已保存为 .parquet 格式！")

Creating parquet from Arrow format: 100%|██████████| 21/21 [00:00<00:00, 2978.51ba/s]

数据集已保存为 .parquet 格式！


In [ ]:
import json
with open('path/to/scienceqa/train.json','r') as f:
    data = json.load(f)

with open('path/to/scienceqa/test.json','r') as f:
    data_test = json.load(f)

Generating train split: 8639 examples [00:00, 129117.36 examples/s]


Dataset({
    features: ['images', 'problem', 'answer'],
    num_rows: 8639
})

In [80]:
from PIL import Image as PILImage

vqa = []

for item in data:
    if 'image' in item.keys():
        problem = item['conversations'][0]['value']
        images = [item['image'].replace('ScienceQA/images/train','ScienceQA/train')]
        answer = item['conversations'][-1]['value']
        messages = [{
            'role': 'user',
            'content': problem 
        }, {
            'role': 'assistant',
            'content': answer
        }]
        vqa.append({
            'messages': messages,
            'images': images,
        })

In [81]:
vqa_test = []

for item in data_test:
    if 'image' in item.keys():
        problem = '<image>\n'+item['text'].replace('Context: ','')
        images =[item['image'].replace('ScienceQA/images/test','ScienceQA/test')]
        answer = item['answer']
        messages = [{
            'role': 'user',
            'content': problem 
        }, {
            'role': 'assistant',
            'content': answer
        }]
        vqa_test.append({
            'messages': messages,
            'images': images,
        })

In [82]:
vqa[:5]

[{'messages': [{'role': 'user',
    'content': "<image>\nWhich of these states is farthest north?\nA. West Virginia\nB. Louisiana\nC. Arizona\nD. Oklahoma\nAnswer with the option's letter from the given choices directly."},
   {'role': 'assistant', 'content': 'A'}],
  'images': ['ScienceQA/train/1/image.png']},
 {'messages': [{'role': 'user',
    'content': "<image>\nContext: The passage below describes an experiment. Read the passage and then follow the instructions below.\n\nTom placed a ping pong ball in a catapult, pulled the catapult's arm back to a 45° angle, and launched the ball. Then, Tom launched another ping pong ball, this time pulling the catapult's arm back to a 30° angle. With each launch, his friend Justin measured the distance between the catapult and the place where the ball hit the ground. Tom and Justin repeated the launches with ping pong balls in four more identical catapults. They compared the distances the balls traveled when launched from a 45° angle to the dis

In [83]:
vqa_test[:5]

[{'messages': [{'role': 'user',
    'content': "<image>\nPeople can use the engineering-design process to develop solutions to problems. One step in the process is testing if a potential solution meets the requirements of the design.\nThe passage below describes how the engineering-design process was used to test a solution to a problem. Read the passage. Then answer the question below.\n\nGordon was an aerospace engineer who was developing a parachute for a spacecraft that would land on Mars. He needed to add a vent at the center of the parachute so the spacecraft would land smoothly. However, the spacecraft would have to travel at a high speed before landing. If the vent was too big or too small, the parachute might swing wildly at this speed. The movement could damage the spacecraft.\nSo, to help decide how big the vent should be, Gordon put a parachute with a 1 m vent in a wind tunnel. The wind tunnel made it seem like the parachute was moving at 200 km per hour. He observed the pa

In [84]:
print(len(vqa))
print(len(vqa_test))

6218
2017
